In [1]:
import torch
from torch.nn import RNN

import pandas as pd

import airsim

In [2]:
data_train = pd.read_csv('test.csv')

In [3]:
data_train.shape

(1608, 10)

In [4]:
data_train.head()

,Unnamed: 0,load_time,radar_system_id,detection_time,detection_ao_id,detection_id,detection_error,detection_x,detection_y,detection_z
0,0,0.0,0.0,0.0,0,0,10,5006.423273,-6.618572,10064.789211
1,1,0.0,0.0,0.0,1,1,10,-4.241559,5011.932936,9984.840969
2,2,0.0,0.0,0.0,2,2,10,13.506236,-5006.094006,9961.920517
3,3,0.0,0.0,0.0,3,3,10,-4984.958634,-0.291319,10004.021689
4,0,0.0,1.0,0.0,0,0,10,4997.609230,-13.314041,10063.979486


In [11]:
E, H  = 2, 3
B, L  = 4, 5

rnn = RNN(E, H)

In [19]:
for k, v in rnn.state_dict().items():
    print(f'{k:12s} : {tuple(v.shape)}')

weight_ih_l0 : (3, 2)
weight_hh_l0 : (3, 3)
bias_ih_l0   : (3,)
bias_hh_l0   : (3,)


In [4]:
X  = torch.rand(L, B, E)
Y, Hn = rnn(X)
print(tuple(Y.shape), tuple(Hn.shape))

(5, 4, 3) (1, 4, 3)


In [5]:
W_ih, W_hh = rnn.weight_ih_l0.detach(), rnn.weight_hh_l0.detach()
B_ih, B_hh = rnn.bias_ih_l0.detach(),   rnn.bias_hh_l0.detach()

In [6]:
Hn = torch.zeros(B,H)

for x in X:
    Hn =torch.tanh(  torch.addmm(B_ih, x,  W_ih.t()) 
                   + torch.addmm(B_hh, Hn, W_hh.t()) )
    print(Hn)

tensor([[-0.4665, -0.5514,  0.4214],
        [-0.5177, -0.7107,  0.2671],
        [-0.5061, -0.6985,  0.1490],
        [-0.4652, -0.6110,  0.0387]])
tensor([[-0.3184, -0.8571,  0.4663],
        [-0.2735, -0.8709,  0.3482],
        [-0.2609, -0.8453,  0.3728],
        [-0.2420, -0.7918,  0.1533]])
tensor([[-0.3116, -0.8174,  0.7222],
        [-0.3775, -0.8796,  0.5908],
        [-0.2790, -0.7193,  0.7300],
        [-0.3717, -0.8559,  0.3738]])
tensor([[-0.2909, -0.7784,  0.7681],
        [-0.2676, -0.8084,  0.7210],
        [-0.3776, -0.8489,  0.7490],
        [-0.3240, -0.8775,  0.4747]])
tensor([[-0.3714, -0.8669,  0.7030],
        [-0.3683, -0.8531,  0.7403],
        [-0.3252, -0.8854,  0.5531],
        [-0.2785, -0.7842,  0.7161]])


In [7]:
Hn = torch.zeros(1,B,H)
for x in X:    
    _, Hn = rnn( x.view(1,B,E), Hn )
    print(Hn)

tensor([[[-0.4665, -0.5514,  0.4214],
         [-0.5177, -0.7107,  0.2671],
         [-0.5061, -0.6985,  0.1490],
         [-0.4652, -0.6110,  0.0387]]], grad_fn=<StackBackward0>)
tensor([[[-0.3184, -0.8571,  0.4663],
         [-0.2735, -0.8709,  0.3482],
         [-0.2609, -0.8453,  0.3728],
         [-0.2420, -0.7918,  0.1533]]], grad_fn=<StackBackward0>)
tensor([[[-0.3116, -0.8174,  0.7222],
         [-0.3775, -0.8796,  0.5908],
         [-0.2790, -0.7193,  0.7300],
         [-0.3717, -0.8559,  0.3738]]], grad_fn=<StackBackward0>)
tensor([[[-0.2909, -0.7784,  0.7681],
         [-0.2676, -0.8084,  0.7210],
         [-0.3776, -0.8489,  0.7490],
         [-0.3240, -0.8775,  0.4747]]], grad_fn=<StackBackward0>)
tensor([[[-0.3714, -0.8669,  0.7030],
         [-0.3683, -0.8531,  0.7403],
         [-0.3252, -0.8854,  0.5531],
         [-0.2785, -0.7842,  0.7161]]], grad_fn=<StackBackward0>)
